In [1]:
#Import of Python available libraries
import sculib

#mpi = sculib.scu()
#mpi.ip = '10.96.64.10'
sim = sculib.scu()
sim.ip = '10.96.254.85'

In [2]:
def mpi_t6102b2(scu):

    #Feed Indexer tests [316-000000-043] 
    print('Test 6.10.2-b2')

    '''
        e. Feed Indexer: Stability - gravity, continuous, cycling (6.10.2-b2)
        (Note this is similar to 6.10.2-b1 with additional FI motions)
        1) Manually move the Dish to 15° Elevation, any Azimuth angle and FI 
        at SPF Band5c and de-activate all axes
        2) Manually start the LDS data recording
        3) Activate the Elevation axis
            ◦ wait for 15 seconds to complete the activation process (note the current SPF Band position(1))
        4) command the FI to the SPF B1 position,
            ◦  wait for 40 seconds to complete the move
        5) Command the FI to the SPF B2 position, 
            ◦ wait for 40 seconds to complete the move
        6) Command the FI to the initial position(1),
            ◦ wait for 40 seconds to complete the move
        7) Start the ACU data recording on the SCU using logging configuration TBD,
            ◦ wait for 60 seconds to stabilise
        8) Command elevation upwards to 85° at a limited rate of 0.1°/s,
            ◦ wait for 760 seconds to complete the move and to stabilise at  85°
        9) Command elevation downwards to 15° at a limited rate of 0.1°/s,
            ◦  wait for 760 seconds to complete the move and to stabilise at 15°
        10) Command the FI to the SPF B1 position,
            ◦ wait for 40 seconds to complete the move
        11) Command the FI to the SPF B2 position,
            ◦ wait for 40 seconds to complete the move
        12) Command the FI to the initial position(1),
            ◦ wait for 100 seconds to complete the move and to stabilise
        13) Repeat steps 8) and 12) for a total of 5 cycles
        14) Stop the ACU data recording
        15) De-activate the Elevation axis
        16) Extract the data on the SCU @ 5Hz sampling
        17) Rename the file to TBD (eg. 6p10p2b2_i.csv)
        18) Manually stop the LDS data recording
        19) Repeat the sequence 1) to 18)  with the FI at SPF Band 1 starting position
        20) Repeat the sequence 1) to 18)  with the FI at SPF Band 2 starting position

    (Total duration ~ 2h30min per SPF Band => ~7h30min)

    (1) The script to sample the current SPF Band position at the start of the script and use it as the “applicable SPF Band” since the test will be repeated for different SPF Band positions
    '''
    #Feed Indexer stability: Henk email 2) iv.

    #Revisions:
    #0.1 initial rewrite
    #0.2 rewrite for use with sculib

    #Common setup
    config_name = 'HN_INDEX_TEST'

    loop1 = 5
    
    bands_seq1 = [ 'Band 5c', 'Band 1', 'Band 2']
    bands_seq2 = [ 'Band 1', 'Band 2', 'Band 5c']
    bands_seq3 = [ 'Band 2', 'Band 5c', 'Band 1']
    bands = bands_seq1

    #az_start = 0
    az_start = -180
    #el_start = 15
    el_start = 20
    el_speed = 0.1
    #elevations = [85, 50, 15]
    elevations = [85, 50, 20] #to avoid lower limit issue

    wait5 = 5
    wait10 = 10
    wait100 = 100
    sampling = 200
    scu.debug = False
    #test only remove these
    #'''
    loop1 = 1
    wait5 = 1
    wait10 = 1
    wait100 = 1
    sampling = 5000
    scu.debug = True
    #'''

    #Step 1
    scu.interlock_acknowledge_dmc()
    scu.wait_duration(wait10*2)

    scu.activate_dmc()
    scu.wait_duration(wait10*3)

    #Move to starting az
    scu.abs_azimuth(az_start, az_speed)
    scu.wait_duration(wait10)

    #Move to starting el
    scu.abs_elevation(el_start, el_speed)
    scu.wait_duration(wait10*2)

    #Move to starting band
    scu.move_to_band(bands[0])
    scu.wait_duration(wait10*5)

    #deactivate_dmc()

    #Step 2a
    print('Start LDS recording here:')
    #input()

    #Step 2b
    if scu.logger_state() == 'RECORDING':
        print('WARNING, already recording - attempting to stop and start a fresh logger')
        scu.stop_logger()  
        scu.wait_duration(wait5)
    if scu.logger_state() == 'STOPPED':
        print('Starting logger with config: {}'.format(config_name))
        scu.start_logger(config_name)
        scu.wait_duration(wait5)

    #Step 3
    # activate elevation axis?
    #activate_el()


    #Step 4
    band=bands[1]
    scu.move_to_band(band)
    scu.wait_duration(wait10*6)

    #Step 5
    band=bands[2]
    scu.move_to_band(band)
    scu.wait_duration(wait10*6)

    #Step 6
    band = bands[0]
    scu.move_to_band(band)
    scu.wait_duration(wait100)

    #Step 7
    #recording already started

    i=0
    while i < loop1:
        i+=1
        print('Loop 1 - Step 8 to 9: {:.0f} of {:.0f} '.format(i, loop1,))

        #Step 8
        el_angle = 85
        el_speed = 0.1
        scu.abs_elevation(el_angle, el_speed)
        scu.wait_duration(wait100*8)

        #Step 9
        #el_angle = 15
        el_angle = 20
        el_speed = 0.1
        scu.abs_elevation(el_angle, el_speed)
        scu.wait_duration(wait100*8)
        #Step 10 loop end

        #Step 10
        band=bands[1]
        scu.move_to_band(band)
        scu.wait_duration(wait10*6)

        #Step 11
        band=bands[2]
        scu.move_to_band(band)
        scu.wait_duration(wait10*6)

        #Step 12
        band = bands[0]
        scu.move_to_band(band)
        scu.wait_duration(wait10*6)


    #Step 13
    #stop recording
    scu.stop_logger()
    scu.wait_duration(wait5)
    print(scu.logger_state())

    #Step 14
    #Stop acu recording
    #request elevation axis only?
    #deactivate_dmc()

    #Step 15
    #deactivate_el()

    #Step 16
    print("Extract the data on the SCU @ 5Hz sampling")

    #Step 17
    print("Rename the file to TBD (eg. 6p10p2b2_i.csv)")
    scu.save_session('6p10p2b2_i_' + bands[0], interval_ms = sampling)

    #Step 18
    print("Stop LDS data recording")

    #19) Repeat the sequence 1) to 18)  with the FI at SPF Band 1 starting position
    #20) Repeat the sequence 1) to 18)  with the FI at SPF Band 2 starting position

    scu.deactivate_dmc()
    scu.wait_duration(wait10*2)


In [3]:
#mpi_t6102b(mpi)
mpi_t6102b2(sim)


Test 6.10.2-b2
reset dmc


KeyboardInterrupt: 